#Interactive Proteus parallel example

##Load IPython support for working with MPI tasks

In [1]:
from IPython.parallel import Client
c = Client(profile="mpi")
view = c[:]
view.activate()

##Load interactive Proteus module, physics, and numerics on the MPI tasks and solve problem

In [2]:
%%px --block
# perform imports on engines
from proteus.iproteus import *
from proteus import default_n,default_s,default_so
import ladr_2d_p,ladr_2d_n
ladr_2d_n.nnx = 201
ladr_2d_n.nny = 201
nOutputSteps = 301
ladr_2d_n.tnList = [0.25*float(i)/float(nOutputSteps-1) for i in range(nOutputSteps)]
from proteus import Comm

In [3]:
def compute_triangles():
    # store x,y,u,triangles in global engine namespace
    global x,y,u,triangles
    comm = Comm.get()
    pList = [ladr_2d_p]
    nList = [ladr_2d_n]
    so = default_so
    so.name = pList[0].name = "ladr_2d"
    so.sList=[default_s]
    so.tnList = ladr_2d_n.tnList
    print ladr_2d_n.tnList,ladr_2d_n.nnx,ladr_2d_n.nny
    nList[0].multilevelLinearSolver=default_n.KSP_petsc4py
    #opts.viewer='vtk'
    #viewers = Viewers.viewerOn(so.name+`comm.rank()`,opts.viewer)
    ns = NumericalSolution.NS_base(so,pList,nList,so.sList,opts)
    ns.calculateSolution('run1')
    x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,0]
    y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,1]
    triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
    u = ns.modelList[0].levelModelList[-1].u[0].dof

    #if comm.isMaster():
    #    imageList = [window.png for window in viewers.windowDict.values()]
    #else:
    #    imageList = []
    
    # return the length of x
    return len(x)

Sync with the engines, to make sure they are ready:

In [4]:
view.apply_sync(lambda : 'ready')

['ready', 'ready', 'ready', 'ready']

Submit the execution asynchronously

In [5]:
compute_ar = view.apply_async(compute_triangles)

In [6]:
x_ar = view.gather('x')
y_ar = view.gather('y')
u_ar = view.gather('u')
tri_ar = view.pull('triangles')

now actually wait for the results:

In [7]:
ns,x,y,u,triangles = [ ar.get() for ar in (compute_ar, x_ar, y_ar, u_ar, tri_ar) ]

Compute the timings for execution and IPython overhead

In [8]:
compute_request = min(compute_ar.submitted) # exectution request sent
compute_start = min(compute_ar.started) # first MPI execution started
compute_complete = max(compute_ar.completed) # last MPI execution completed
data_received = max(tri_ar.received) # last bit of results arrived back in the Client


compute = (compute_complete-compute_start).total_seconds()
with_request = (compute_complete-compute_request).total_seconds()
roundtrip = (data_received - compute_request).total_seconds()
print "computation time: %.3f" % compute
print "   from request : %.3f" % with_request
print "total roundtrip : %.3f" % roundtrip

overhead_ms = 1000*(roundtrip-compute)
compute_ms = 1000*(compute)
print "IPython overhead: %i/%i ms = %.2f%%" % (overhead_ms, compute_ms, 100*(overhead_ms / compute_ms))

computation time: 398.615
   from request : 398.617
total roundtrip : 401.267
IPython overhead: 2652/398614 ms = 0.67%


##Combine subdomain solutions and plot

In [9]:
# shift triangles by preceding lens
shifts = numpy.cumsum([0]+ns[:-1])
flat_triangles = numpy.concatenate( [ tri + shift for tri,shift in zip(triangles,shifts) ] )

tricontourf(x,y,flat_triangles,u)

NameError: name 'numpy' is not defined

In [ ]:
#print view['imageList']
#png = view['imageList'][0][0]
#print png

In [ ]:
#from IPython.core.display import Image
#Image(png,embed=True)